In [ ]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [ ]:
from django.db import connection
connection.connect()

In [ ]:
from fotoweb.models import *

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

class SelDriver():
  def __init__(self,dbg=False):
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    if dbg:
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    #Change chrome driver path accordingly
    #chrome_driver = "chromedriver.exe
    #chrome_driver = r"c:\programs\chrome-win64"
    service = Service()
    self.driver = webdriver.Chrome(service=service, options=chrome_options)
    print (self.driver)
    self.driver.implicitly_wait(5)
    self.vars = {}

In [ ]:
driver=SelDriver(False)

In [ ]:

#ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee

In [ ]:
srcs=[]
for e in ee:
    src=e.get_attribute('src')
    r=re.search(r'(\d+)\.jpg',src)
    print(src,r.group(1))
    srcs.append(r.group(1))



In [ ]:
images=[]
for src in srcs:
    driver.driver.get('https://submit.shutterstock.com/catalog_manager/images/'+src)
    name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text
    description = driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
    images.append({
        'name':name,
        'description':description,
        'shutter-id':src
    })
    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text.replace('.jpg','')
        try:
            i=Image.objects.filter(name__contains=name).first()
            print(name,'found by contains')
        except:
            print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        i.title=driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
        i.shutter=True
        i.shutter_url='https://submit.shutterstock.com/catalog_manager/images/'+src
        i.save()
        print('shutter updated')


In [ ]:
images

In [ ]:
ii